In [ ]:
%matplotlib widget
from ipywidgets import *
import sys
sys.path.append('../..')

import navbench as nb
import matplotlib.pyplot as plt
import numpy as np

def show_im(ax, im, cmap='gray'):
    ax.imshow(im, cmap=cmap)
    ax.set_xticks(())
    ax.set_yticks(())

STEP = 1

db_train = nb.Database('../../datasets/rc_car/rc_car_big/unwrapped_20210303_150314', interpolate_xy=True)
db_test = db_train
train_entry = db_train.read_image_entries(0)
test_entry = db_test.read_image_entries(1)

width = train_entry.image.shape[1]
snap = np.roll(train_entry.image, -round(train_entry.heading * width / (2 * np.pi)), axis=1)
ridf = nb.ridf(test_entry.image, snap, step=STEP)

_, ax = plt.subplots(4)
show_im(ax[1], snap)
ax[1].set_title('Training image')
ax[3].plot(range(0, width, STEP), ridf)
lines, = ax[3].plot(0, ridf[0], 'ro')

# halfwidth = round(width / 2)

def update(rotation):
    test_im = np.roll(test_entry.image, rotation, axis=1)
    show_im(ax[0], test_im)
    ax[0].set_title('Test image (rotation=%d)' % rotation)

    diffim = test_im.astype(float) - snap.astype(float)
    diffim /= 255
    mean_abs_diff = np.mean(np.abs(diffim))

    diffim = (diffim + 1) / 2
    show_im(ax[2], diffim, None)
    ax[2].set_title('Difference (mean=%g)' % mean_abs_diff)

    lines.set_xdata(rotation)
    lines.set_ydata(ridf[int(rotation / STEP)])

interact(update, rotation=widgets.IntSlider(0, 0, width - STEP, step=STEP));